# FastF1 Classification Baseline

This notebook loads the dataset and trains a simple logistic regression baseline.

In [1]:
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    f1_score,
    precision_score,
    recall_score,
    average_precision_score,
    confusion_matrix,
)


In [2]:
FEATURES = ['year', 'round', 'avg_race_lap_time_s', 'quali_position', 'prev_points_total']
TARGET = 'scored_points'
data_path = Path('data/grandprix_features.csv')

df_model = pd.read_csv(data_path)

print(f"Rows after cleaning: {len(df_model)}")
df_model.head()

Rows after cleaning: 300


,year,round,event,driver,team,quali_position,avg_race_lap_time_s,finish_position,points_awarded,prev_points_total,scored_points
0,2022,1,Bahrain Grand Prix,LEC,Ferrari,1,97.604208,1.0,26.0,0.0,1
1,2022,1,Bahrain Grand Prix,SAI,Ferrari,3,98.079957,2.0,18.0,0.0,1
2,2022,1,Bahrain Grand Prix,HAM,Mercedes,5,98.266244,3.0,15.0,0.0,1
3,2022,1,Bahrain Grand Prix,RUS,Mercedes,9,98.639022,4.0,12.0,0.0,1
4,2022,1,Bahrain Grand Prix,MAG,Haas F1 Team,7,98.852833,5.0,10.0,0.0,1


In [3]:
X = df_model[FEATURES]
y = df_model[TARGET]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train.shape, X_test.shape

((240, 5), (60, 5))

Pipeline: Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(class_weight='balanced', max_iter=200))])
CV ROC-AUC: 0.805 ± 0.035
CV PR-AUC (avg precision): 0.802 ± 0.076

Holdout classification report:
              precision    recall  f1-score   support

           0       0.76      0.73      0.75        30
           1       0.74      0.77      0.75        30

    accuracy                           0.75        60
   macro avg       0.75      0.75      0.75        60
weighted avg       0.75      0.75      0.75        60

Holdout ROC-AUC: 0.8688888888888888
Holdout PR-AUC (avg precision): 0.8490854562570249

Additional metrics:
Precision: 0.7419354838709677
Recall: 0.7666666666666667
F1 (binary): 0.7540983606557377
F1 (macro): 0.7499305362600722
Confusion matrix (raw):
 [[22  8]
 [ 7 23]]

Confusion matrix as table:


,Pred 0,Pred 1
Actual 0,22,8
Actual 1,7,23


In [4]:
# Logistic regression baseline (imputer + scaler)
log_reg = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler(),
    LogisticRegression(max_iter=200, class_weight='balanced'),
)
print("Pipeline:", log_reg)

# Cross-validation: ROC-AUC and PR-AUC (average_precision)
cv_auc = cross_val_score(log_reg, X_train, y_train, cv=5, scoring='roc_auc')
cv_pr = cross_val_score(log_reg, X_train, y_train, cv=5, scoring='average_precision')
print(f'CV ROC-AUC: {cv_auc.mean():.3f} ± {cv_auc.std():.3f}')
print(f'CV PR-AUC (avg precision): {cv_pr.mean():.3f} ± {cv_pr.std():.3f}')

# Fit on training data
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)
y_proba = log_reg.predict_proba(X_test)[:, 1]

# Holdout evaluation
print('\nHoldout classification report:')
print(classification_report(y_test, y_pred))
print('Holdout ROC-AUC:', roc_auc_score(y_test, y_proba))
print('Holdout PR-AUC (avg precision):', average_precision_score(y_test, y_proba))
print('\nAdditional metrics:')
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 (binary):', f1_score(y_test, y_pred))
print('F1 (macro):', f1_score(y_test, y_pred, average="macro"))

# Confusion matrix (numeric) and visualization as table + heatmap
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix (raw):\n', cm)

# Visualization: DataFrame table and seaborn heatmap
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

cm_df = pd.DataFrame(
    cm,
    index=[f'Actual {i}' for i in range(cm.shape[0])],
    columns=[f'Pred {i}' for i in range(cm.shape[1])],
)

print('\nConfusion matrix as table:')
display(cm_df)

plt.figure(figsize=(6, 4))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()
